<h1 align="center">1. Базовые понятия статистики</h1>

<h3 align="center">Задание 1. Загрузка данных</h3>

Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import seaborn as sns
from scipy import stats
#from pandas_profiling import ProfileReport

In [ ]:
req = requests.get('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names')
soup = BeautifulSoup(req.text, 'html.parser')
info = str(soup)
soup

In [ ]:
cols = ['surgery?', 'Age', 'rectal_temperature', 'pulse', 'respiratory_rate','temperature_of_extremities', 'pain', 'outcome']

df = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv',
                 header=None, na_values='?', usecols = [0, 1, 3, 4, 5, 6, 10, 22], names = cols)
df.info()
df

In [ ]:
df.describe()

### Вариант с использованием библиотеки Pandas Profiling для быстрого решения всех заданий

In [ ]:
cols = ['surgery?', 'Age', 'rectal_temperature', 'pulse', 'respiratory_rate','temperature_of_extremities', 'pain', 'outcome']
df.drop(cols, axis=1,inplace=True)

profile = ProfileReport(df, title="Pandas Profiling Report")

profile.to_widgets()

![](Pandas_profiling.png)

<h3 align="center">Задание 2. Первичное изучение данных</h3>

Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

- в столбце Age есть значения 9, которые лежат вне интервала возможных значений (1,2), предположим, что это неправильное распознавание значения 2, так как его в датафрейме изначально нет, заменим 9 на на 2.


- 'rectal_temperature', 'pulse', 'respiratory_rate' - количественные, рассчитаем базовые статистики


- 'surgery?', 'Age','temperature_of_extremities', 'pain', 'outcome' - категориальные, рассчитаем частотный анализ и критерии сопоставления групп, проверим уникальные значения категорий на предмет ошибок.

In [ ]:
df.loc[(df.Age == 9), 'Age'] = 2
df

### Первый вариант поиска базовых статистик (простой), без разделения по типам данных

In [ ]:
df_max = df.select_dtypes(include='number').max()
df_min = df.select_dtypes(include='number').min()
df_range = df_max - df_min
df_mean = df.select_dtypes(include='number').mean()
df_mode = df.select_dtypes(include='number').mode()
df_median = df.select_dtypes(include='number').median()
df_std = df.select_dtypes(include='number').std()
df_var = df.select_dtypes(include='number').var().round(2)

df_quantile_1_3 = df.select_dtypes(include='number').quantile([0.25, 0.75])
df_quantile_random = df.select_dtypes(include='number').quantile(0.33)
df_quantile_1 = df.select_dtypes(include='number').quantile(0.25)
df_quantile_3 = df.select_dtypes(include='number').quantile(0.75)
iqr = df_quantile_3 - df_quantile_1

lower_bound = df_quantile_1 - (1.5 * iqr) 
upper_bound = df_quantile_3 + (1.5 * iqr)

print('МАКСИМУМ', '\n', df_max, '\n', '\n', 'МИНИМУМ', '\n', df_min,  '\n', '\n', 'РАЗМАХ', '\n', df_range,  
      '\n', '\n', 'СРЕДНЕЕ АРИФМЕТИЧЕСКОЕ', '\n', df_mean, '\n', '\n','МЕДИАНА', '\n', df_median,  '\n', '\n', 
      'СТАНДАРТНОЕ ОТКЛОНЕНИЕ', '\n', df_std, '\n', '\n', 'ДИСПЕРСИЯ', '\n', df_var, '\n',  '\n', 'МОДА', '\n', df_mode, 
      '\n',  '\n', 'ПЕРВЫЙ КВАРТИЛЬ', '\n', df_quantile_1, '\n',  '\n', 'ТРЕТИЙ КВАРТИЛЬ', '\n', df_quantile_3, 
      '\n',  '\n', 'ПРОИЗВОЛЬНЫЙ ПЕРЦЕНТИЛЬ', '\n', df_quantile_random, '\n',  '\n', 'МЕЖКВАРТИЛЬНЫЙ РАЗМАХ', '\n', iqr)

### Максимум и минимум

In [ ]:
print(max(df['rectal_temperature']))
print(min(df['pulse']))
print(min(df['respiratory_rate']))

### Размах
- разность между наибольшим и наименьшим значениями показателя

In [ ]:
rect_range = df['rectal_temperature'].max() - df['rectal_temperature'].min()
pulse_range = df['pulse'].max() - df['pulse'].min()
resp_range = df['respiratory_rate'].max() - df['respiratory_rate'].min()
print(f'Rectal Temperature range: {rect_range}')
print(f'Pulse range: {pulse_range}')
print(f'Respiratory Rate range: {resp_range}')

### Среднее арифметическое
- частное от деления суммы всех чисел ряда на их количество

In [ ]:
print(df['rectal_temperature'].mean())
print(df['pulse'].mean())
print(df['respiratory_rate'].mean())

### Медиана
- число, половина из элементов выборки больше которого, а другая половина – меньше

In [ ]:
print(df['rectal_temperature'].median())
print(df['pulse'].median())
print(df['respiratory_rate'].median())

### СКО
- измеряется в единицах самой случайной величины и используется при расчете стандартной ошибки среднего арифметического, при построении доверительных интервалов, при статистической проверке гипотез, при измерении линейной взаимосвязи между случайными величинами. Является корнем из дисперсии.

In [ ]:
print(df['rectal_temperature'].std())
print(df['pulse'].std())
print(df['respiratory_rate'].std())

### Дисперсия
- просто квадрат стандартного отклонения. Во многих статистических формулах удобнее использовать СКО, а не извлекать каждый раз  корень из дисперсии.

In [ ]:
print(df['rectal_temperature'].var())
print(df['pulse'].var())
print(df['respiratory_rate'].var())

### Первый и третий квантили

Меры разброса характеризуют степень индивидуальных отклонений величины от среднего.
Квантили (процентили) - это значение, ниже (выше) которого попадает определенный процент наблюдений отсортированной выбокрке.

- 0.25-квантиль называется первой (или нижней) квартилью
- 0.5-квантиль называется второй квартилью (это же тоже самое, что медиана!)
- 0.75-квантиль называется третьей (или верхней)  квартилью.


In [ ]:
print(df['rectal_temperature'].quantile([0.25, 0.75]))
print(df['pulse'].quantile([0.25, 0.75]))
print(df['respiratory_rate'].quantile([0.25, 0.75]))

### Произвольный перцентиль

In [ ]:
print(df['rectal_temperature'].quantile(0.33))
print(df['pulse'].quantile(0.33))
print(df['respiratory_rate'].quantile(0.33))

### Межквартильный размах
- это разница между 1-м и 3-м квартилями, т.е. между 25-м и 75-м процентилями.

In [ ]:
Q_1_rect = df['rectal_temperature'].quantile(0.25)
Q_3_rect = df['rectal_temperature'].quantile(0.75)
IQR_rect = Q_3_rect - Q_1_rect
print(IQR_rect.round(2))

Q_1_pulse = df['pulse'].quantile(0.25)
Q_3_pulse = df['pulse'].quantile(0.75)
IQR_pulse = Q_3_pulse - Q_1_pulse
print(IQR_pulse)

Q_1_resp = df['respiratory_rate'].quantile(0.25)
Q_3_resp = df['respiratory_rate'].quantile(0.75)
IQR_resp = Q_3_resp - Q_1_resp
print(IQR_resp)

### Посмотрим на диаграммы количественных столбцов

In [ ]:
sns.histplot(x = 'rectal_temperature', data = df)

In [ ]:
sns.histplot(x = 'pulse', data = df)

In [ ]:
sns.histplot(x = 'respiratory_rate', data = df)

### Выбросы
Выбросы (outliers) результаты измерения, сильно выделяющиеся в общей выборке. Наиболее простой из методов обнаружения выбросов основан на межквартильном размахе (т.е. все, что не попадает в указанные диапазоны, является выбросом).

Наиболее распространенные причины выбросов в наборе данных:
- Ошибки ввода данных (человеческий фактор)
- Погрешности измерения (ошибки приборов)
- Преднамеренное (например, сделанные для проверки методов обнаружения оборудованием)
- Ошибки обработки данных
- Ошибки выборки (извлечение или смешивание данных из неправильных или различных источников)
- Естественные выбросы (не ошибки, а реальные исключительные наблюдения в данных)

In [ ]:
lower_bound_rect = Q_1_rect - (1.5 * IQR_rect).round(2)
upper_bound_rect = Q_3_rect + (1.5 * IQR_rect).round(2)

lower_bound_pulse = Q_1_pulse - (1.5 * IQR_pulse) 
upper_bound_pulse = Q_3_pulse + (1.5 * IQR_pulse)

lower_bound_resp = Q_1_resp - (1.5 * IQR_resp) 
upper_bound_resp = Q_3_resp + (1.5 * IQR_resp)

remove_outliers = df[df['rectal_temperature'].between(lower_bound_rect, upper_bound_rect, inclusive=True)]
remove_outliers

In [ ]:
remove_outliers_1 = df[df['pulse'].between(lower_bound_rect, upper_bound_rect, inclusive=True)]
remove_outliers_1

In [ ]:
remove_outliers_2 = df[df['respiratory_rate'].between(lower_bound_rect, upper_bound_rect, inclusive=True)]
remove_outliers_2

In [ ]:
# что это за выбросы?
df[~df['rectal_temperature'].between(lower_bound_rect, upper_bound_rect, inclusive=True)]

### Частотный анализ категориальных столбцов
- Теперь проведём частотный анализ по категориальным столбцам с помощью value_counts() и проверим уникальные значения на предмет ошибок. Результат - внесение изменений не требуется.

In [ ]:
df_surgery = df['surgery?'].value_counts()
print(df_surgery, '\n')

df_Age = df['Age'].value_counts()
print(df_Age, '\n')

df_t = df['temperature_of_extremities'].value_counts()
print(df_t, '\n')

df_pain = df['pain'].value_counts()
print(df_pain, '\n')

df_outcome = df['outcome'].value_counts()
print(df_outcome, '\n')

### Посмотрим на гистограммы категориальных столбцов

In [ ]:
sns.countplot(x = 'surgery?', data = df.sort_values('surgery?'))

In [ ]:
sns.countplot(x = 'Age', data = df)

In [ ]:
sns.histplot(x = 'temperature_of_extremities', data = df)

In [ ]:
sns.countplot(x = 'pain', data = df)

In [ ]:
sns.countplot(x = 'outcome', data = df)

<h3 align="center">Задание 3. Работа с пропусками</h3>

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [ ]:
df.isna().sum()

In [ ]:
df.info()

### Гипотеза
- Проверим взаимосвязь между столбцами pain и outcome с группировкой по возрасту лошади. Можно сделать вывод, что молодые лошади при коликах погибают. Взрослые лошади при низком уровне боли (до 4) остются живыми, а с 4 уровня боли, погибают все лошади. Признаем эту гипотезу допустимой, поэтому недостающие значения в столбце outcome заполняеем исходя из значения столбца pain.

In [ ]:
print(df.groupby(['pain','Age'])['outcome'].agg(pd.Series.mode))

- Заполним недостающими значениями столбец outcome, прежде создав копию.

In [ ]:
fill_df = df
fill_df['outcome'].fillna(fill_df.groupby(['pain','Age'])['outcome'].transform('median'), inplace=True)
fill_df.info()

### Гипотеза
- Проверим взаимосвязь между столбцами surgery?(операция) и pain (боль)

In [ ]:
print(fill_df.groupby(['surgery?'])['pain'].median())

- Связь есть - операция была (1) при высоком медианном значении боли (3), и не было (2) при низком значении боли (2).
- Заполним недостающие значениями столбца pain на сонове медианных значений surgery

In [ ]:
fill_df['pain'].fillna(fill_df.groupby(['surgery?'])['pain'].transform('median'), inplace=True)
fill_df.info()

- В подтверждение гипотезы проверим обратную взаимосвязь между столбцами pain и surgery

In [ ]:
print(fill_df.groupby(['pain'])['surgery?'].median())

- Связь есть, на уровнях боли 3 и выше, было хирургическое вмешательство. Восстановим отсутствующее значение в столбце 'surgery?'

In [ ]:
fill_df['surgery?'].fillna(fill_df.groupby(['pain'])['surgery?'].transform('median'), inplace=True)
fill_df.info()

### Гипотеза
- Проверим взаимосвязь между outcome  и temperature_of_extremities

In [ ]:
print(df.groupby(['outcome'])['temperature_of_extremities'].median())

- Выявлена связь, в случаях смертности (outcome = 2 или 3), температура конечностей (temperature of extremities) всегда была 3 (это признак шока). В случае если outcome = 1 (лошадь осталась живой), температура конечностей = 2 (норма).
- Восстановим отсутствующие значения в столбце 'temperature_of_extremities'

In [ ]:
fill_df['temperature_of_extremities'].fillna(fill_df.groupby(['outcome'])['temperature_of_extremities'].transform('median'), inplace=True)
fill_df.info()

### Гипотеза
- Проверим взаимосвязь между столбцами temperature_of_extremities и pulse

In [ ]:
print(df.groupby(['temperature_of_extremities'])['pulse'].median())

- Выявлена связь, чем ниже температура конечностей (признак заболевания), тем выше пульс.
- Восстановим отсутствующие значения в столбце 'pulse' 

In [ ]:
fill_df['pulse'].fillna(fill_df.groupby(['temperature_of_extremities'])['pulse'].transform('median'), inplace=True)
fill_df.info()

### Гипотеза
- Проверим утверждения из описания, что ректальная температура повышается во время инфекции, а во время болевого шока (значение - pain) температура понижается

In [ ]:
print(df.groupby(['pain'])['rectal_temperature'].median())

- Гипотезу можно признать успешной. При высоких показателях боли, медианная температура понижается.
- Восстановим отсутствующие значения в столбце 'rectal_temperature' 

In [ ]:
fill_df['rectal_temperature'].fillna(fill_df.groupby(['pain'])['rectal_temperature'].transform('median'), inplace=True)
fill_df.info()

- В описании respiratory_rate, написано что влияние этого показателя вызывает сомнения, из-за существенных колебаний значений.
- Заполним недостающие значения медианными.

In [ ]:
fill_df['respiratory_rate'].fillna(fill_df['respiratory_rate'].median(), inplace=True)
fill_df.info()

In [ ]:
df.isna().sum()

### Статистики после заполнения

In [ ]:
fill_df_max = fill_df.select_dtypes(include='number').max()
fill_df_min = fill_df.select_dtypes(include='number').min()
fill_df_range = fill_df_max - fill_df_min
fill_df_mean = fill_df.select_dtypes(include='number').mean()
fill_df_mode = fill_df.select_dtypes(include='number').mode()
fill_df_median = fill_df.select_dtypes(include='number').median()
fill_df_std = fill_df.select_dtypes(include='number').std()
fill_df_var = fill_df.select_dtypes(include='number').var().round(2)

fill_df_quantile_1_3 = fill_df.select_dtypes(include='number').quantile([0.25, 0.75])
fill_df_quantile_random = fill_df.select_dtypes(include='number').quantile(0.33)
fill_df_quantile_1 = fill_df.select_dtypes(include='number').quantile(0.25)
fill_df_quantile_3 = fill_df.select_dtypes(include='number').quantile(0.75)
fill_iqr = fill_df_quantile_3 - fill_df_quantile_1

fill_lower_bound = fill_df_quantile_1 - (1.5 * iqr) 
fill_upper_bound = fill_df_quantile_3 + (1.5 * iqr)

print('МАКСИМУМ', '\n', fill_df_max, '\n', '\n', 'МИНИМУМ', '\n', fill_df_min,  '\n', '\n', 'РАЗМАХ', '\n', fill_df_range,  
      '\n', '\n', 'СРЕДНЕЕ АРИФМЕТИЧЕСКОЕ', '\n', fill_df_mean, '\n', '\n','МЕДИАНА', '\n', fill_df_median,  '\n', '\n', 
      'СТАНДАРТНОЕ ОТКЛОНЕНИЕ', '\n', fill_df_std, '\n', '\n', 'ДИСПЕРСИЯ', '\n', fill_df_var, '\n',  '\n', 'МОДА', '\n', fill_df_mode, 
      '\n',  '\n', 'ПЕРВЫЙ КВАРТИЛЬ', '\n', fill_df_quantile_1, '\n',  '\n', 'ТРЕТИЙ КВАРТИЛЬ', '\n', fill_df_quantile_3, 
      '\n',  '\n', 'ПРОИЗВОЛЬНЫЙ ПЕРЦЕНТИЛЬ', '\n', fill_df_quantile_random, '\n',  '\n', 'МЕЖКВАРТИЛЬНЫЙ РАЗМАХ', '\n', fill_iqr)

### Посмотрим на графики в получившемся датафрейме

In [ ]:
sns.histplot(x = 'rectal_temperature', data = fill_df)

In [ ]:
sns.histplot(x = 'pulse', data = fill_df)

In [ ]:
sns.histplot(x = 'respiratory_rate', data = fill_df)

In [ ]:
sns.countplot(x = 'surgery?', data = fill_df)

In [ ]:
sns.countplot(x = 'Age', data = fill_df)

In [ ]:
sns.histplot(x = 'temperature_of_extremities', data = fill_df)

In [ ]:
sns.countplot(x = 'pain', data = fill_df)

In [ ]:
sns.countplot(x = 'outcome', data = fill_df)

![](seaborn.jpg)